In [1]:
!pip install PyPortfolioOpt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.1/222.1 kB 9.6 MB/s eta 0:00:00


In [ ]:
# =========================
# Proyecto Quant: Optimización de Cartera con Restricciones Reales
# =========================

# Mount Google Drive to access files
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
from pathlib import Path
from pypfopt import EfficientFrontier, risk_models, expected_returns
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices
import matplotlib.pyplot as plt

# --- Directorios ---
BASE = Path('/content/drive/MyDrive/ProyectoCartera')
PROCESSED_DIR = BASE / 'data' / 'processed'

# --- 1️ Leer y limpiar datos ---
prices = pd.read_csv(PROCESSED_DIR / 'prices.csv', index_col=0, parse_dates=True)
volumes = pd.read_csv(PROCESSED_DIR / 'volumes.csv', index_col=0, parse_dates=True)

# Forzar a numérico y limpiar NaN
prices = prices.apply(pd.to_numeric, errors='coerce').dropna(how='all')
volumes = volumes.apply(pd.to_numeric, errors='coerce').fillna(0)

# --- 2️ Calcular retornos ---
returns = prices.pct_change().dropna(how='all')

# --- 3️ Calcular parámetros esperados ---
mu = expected_returns.mean_historical_return(prices)
S = risk_models.sample_cov(prices)

# --- 4️ Definir cartera y restricciones ---
ef = EfficientFrontier(mu, S, weight_bounds=(0, 1))

# 4a. Límite de liquidez: no más del 30% en activos con poco volumen
n_assets = len(prices.columns)
avg_volumes = volumes.mean()
total_volume = avg_volumes.sum()
max_liquidity_weight = 0.3 * (avg_volumes / total_volume) * n_assets  # escalado
# Pypfopt espera bounds (min, max) por activo
bounds = [(0, max_liquidity_weight[i]) for i in range(n_assets)]
ef.add_objective(lambda w: 0)  # dummy para poder usar custom bounds
ef.bounds = bounds

# 4b. Maximizar Sharpe ratio con restricciones
try:
    raw_weights = ef.max_sharpe(risk_free_rate=0.01)
except Exception as e:
    print(" Error en optimización:", e)
    raw_weights = ef.clean_weights()  # fallback

weights = ef.clean_weights()

# --- 5️ Información de la cartera ---
ret, vol, sharpe = ef.portfolio_performance(verbose=True)

print("\nPesos finales:")
for t, w in weights.items():
    print(f"{t}: {w:.4f}")

# --- 6️ Visualización ---
plt.figure(figsize=(10,5))
plt.bar(weights.keys(), weights.values(), color='skyblue')
plt.xticks(rotation=45, ha='right')
plt.ylabel("Peso en cartera")
plt.title("Pesos de la cartera óptima (restricciones aplicadas)")
plt.tight_layout()
plt.show()

# --- 7️ Discrete Allocation (opcional, para ver cuántas acciones comprar) ---
latest_prices = get_latest_prices(prices)
da = DiscreteAllocation(weights, latest_prices, total_portfolio_value=100000)  # 100k €
allocation, leftover = da.lp_portfolio()
print("\nAsignación discreta (acciones a comprar):")
print(allocation)
print("Dinero sobrante:", leftover)

Mounted at /content/drive
